In [1]:
import numpy as np

from backend.models import (
    SimulationParams,
    UniverseParams,
    FinancingParams,
    StrategyParams,
    OptionsParams,
)
from backend.simulator import (
    generate_ar1_factors_numba,
    generate_correlated_returns_numba,
    generate_prices,
)
from backend.strategy import run_strategy


In [2]:
params = SimulationParams(
    universe=UniverseParams(
        n_assets=50,
        mean_return=6.0,
        volatility=20.0,
        ic=0.05,
        factor_autocorr=0.8,
    ),
    financing=FinancingParams(
        cash_rate=3.0,
        margin_rate=5.0,
        borrow_fee=1.0,
    ),
    strategy=StrategyParams(
        strategy_length=60,
        risk_aversion=4.0,
        long_weight=130.0,
        short_weight=30.0,
        turnover_limit=25.0,
        max_weight=10.0,
        lookback=36,
        transaction_cost_bps=5.0,
    ),
    options=OptionsParams(
        enabled=True,
        call_otm_pct=0.0,
        put_otm_pct=0.0,
        call_alpha_barrier=0,
        put_alpha_barrier=0,
        contract_fee=0.65,
        spread_bps=100.0,
    ),
)


In [3]:
np.random.seed(42)

n_periods = params.strategy.lookback + params.strategy.strategy_length
monthly_mean = (params.universe.mean_return / 100) / 12
monthly_vol = (params.universe.volatility / 100) / np.sqrt(12)

# Heterogeneous asset vols
asset_vol_scalings = np.random.uniform(0.5, 1.5, params.universe.n_assets)
asset_vols = monthly_vol * asset_vol_scalings

# Factors and returns
factor_scores = generate_ar1_factors_numba(
    n_periods,
    params.universe.n_assets,
    params.universe.factor_autocorr,
    42,
)
returns = generate_correlated_returns_numba(
    factor_scores,
    params.universe.ic,
    monthly_mean,
    asset_vols,
    43,
)

# Prices via helper
initial_prices = np.random.uniform(5.0, 200.0, params.universe.n_assets)
prices = generate_prices(returns, initial_prices)


In [4]:
strategy_params = {
    "lookback": params.strategy.lookback,
    "strategy_length": params.strategy.strategy_length,
    "risk_aversion": params.strategy.risk_aversion,
    "long_weight": params.strategy.long_weight,
    "short_weight": params.strategy.short_weight,
    "turnover_limit": params.strategy.turnover_limit,
    "max_weight": params.strategy.max_weight,
    "transaction_cost_bps": params.strategy.transaction_cost_bps,
}

financing_params = {
    "cash_rate": params.financing.cash_rate,
    "margin_rate": params.financing.margin_rate,
    "borrow_fee": params.financing.borrow_fee,
}

options_params = {
    "enabled": params.options.enabled,
    "call_otm_pct": params.options.call_otm_pct,
    "put_otm_pct": params.options.put_otm_pct,
    "call_alpha_barrier": params.options.call_alpha_barrier,
    "put_alpha_barrier": params.options.put_alpha_barrier,
    "contract_fee": params.options.contract_fee,
    "spread_bps": params.options.spread_bps,
}

result = run_strategy(
    returns=returns,
    prices=prices,
    factor_scores=factor_scores,
    strategy_params=strategy_params,
    financing_params=financing_params,
    options_params=options_params,
    verbose=True,
    log_file="detailed_strategy_log.txt",
)

print("✓ Log saved to: detailed_strategy_log.txt")
print(f"✓ Annualized Return: {result['annualized_return']:.2f}%")
print(f"✓ Sharpe Ratio: {result['sharpe_ratio']:.2f}")


✓ Log saved to: detailed_strategy_log.txt
✓ Annualized Return: 3.25%
✓ Sharpe Ratio: 0.47
